In [25]:
from tabula.io import read_pdf
from pandas import read_excel
from difflib import SequenceMatcher

## Realizar leitura do PDF

In [26]:
with open("files/72380.pdf", "rb") as file:
    all_tables = read_pdf(file, pages="all", silent=True, lattice=True)

c:\Python\tabula-pdf\.venv\Lib\site-packages\tabula\io.py:1045: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df[c] = pd.to_numeric(df[c], errors="ignore")
c:\Python\tabula-pdf\.venv\Lib\site-packages\tabula\io.py:1045: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df[c] = pd.to_numeric(df[c], errors="ignore")
c:\Python\tabula-pdf\.venv\Lib\site-packages\tabula\io.py:1045: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df[c] = pd.to_numeric(df[c], errors="ignore")


## Buscar tabelas tratáveis

In [27]:
wanted_tables = [
    "Descrição de Créditos e Descontos do Reclamante",
    "Descrição do Bruto Devido ao Reclamante",
    "Descrição de Débitos do Reclamado por Credor",
]

tables = {}

for table in all_tables:
    common_cols = list(set(table.columns).intersection(wanted_tables))

    if common_cols:
        tables[common_cols[0]] = table

## Ler planilha de DE/PARA

In [28]:
dp = read_excel("files/RPA - De Para.xlsx")
dp["DE"].loc[:] = dp["DE"].str.strip().str.upper()
dp["PARA"].loc[:] = dp["PARA"].str.strip().str.upper()
dp.drop_duplicates("DE", keep="first", inplace=True)

dp["index"] = dp["DE"]
dp.set_index("index", inplace=True)

dp

,DE,PARA
index,,
VERBAS INDENIZATÓRIAS E BENEFÍCIOS,VERBAS INDENIZATÓRIAS E BENEFÍCIOS,VERBAS INDENIZATÓRIAS E BENEFÍCIOS
13º SAL. SOBRE AUX.DOENÇA,13º SAL. SOBRE AUX.DOENÇA,AUXÍLIO DOENÇA
13º SALÁRIO,13º SALÁRIO,VERBAS RESCISÓRIAS
13º SALÁRIO SOBRE ADICIONAL DE HORAS EXTRAS 50%,13º SALÁRIO SOBRE ADICIONAL DE HORAS EXTRAS 50%,HORAS EXTRAS E REFLEXOS
13º SALÁRIO SOBRE ADICIONAL DE INSALUBRIDADE 20%,13º SALÁRIO SOBRE ADICIONAL DE INSALUBRIDADE 20%,ADICIONAL DE INSALUBRIDADE
...,...,...
VALE REFEIÇÃO,VALE REFEIÇÃO,VALE REFEIÇÃO
VALE TRANSPORTE,VALE TRANSPORTE,VALE TRANSPORTE
VALE TRANSPORTE 02/03/2011 A 23/02/2012,VALE TRANSPORTE 02/03/2011 A 23/02/2012,VALE TRANSPORTE


## Selecionar planilha de holerite

In [29]:
filter_rows = [
    "Total"
]

holerite = tables["Descrição do Bruto Devido ao Reclamante"]
holerite = holerite[ ~holerite["Descrição do Bruto Devido ao Reclamante"].isin(filter_rows) ]

holerite["index"] = holerite["Descrição do Bruto Devido ao Reclamante"]
holerite.set_index("index", inplace=True)

holerite

C:\Users\rafin\AppData\Local\Temp\ipykernel_14152\2772191235.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  holerite["index"] = holerite["Descrição do Bruto Devido ao Reclamante"]


,Descrição do Bruto Devido ao Reclamante,Valor Corrigido,Juros,Total
index,,,,
HORAS EXTRAS 50%,HORAS EXTRAS 50%,"24.995,90","715,34","25.711,24"
13o SALÁRIO SOBRE HORAS EXTRAS 50%,13o SALÁRIO SOBRE HORAS EXTRAS 50%,"1.648,55","47,35","1.695,90"
FÉRIAS + 1/3 SOBRE HORAS EXTRAS 50%,FÉRIAS + 1/3 SOBRE HORAS EXTRAS 50%,"1.939,86","55,54","1.995,40"
FGTS 8%,FGTS 8%,"2.054,45","63,15","2.117,60"


## Realizar DE/PARA

In [30]:
df = holerite.join(dp, how="left")
df

,Descrição do Bruto Devido ao Reclamante,Valor Corrigido,Juros,Total,DE,PARA
index,,,,,,
HORAS EXTRAS 50%,HORAS EXTRAS 50%,"24.995,90","715,34","25.711,24",HORAS EXTRAS 50%,HORAS EXTRAS E REFLEXOS
13o SALÁRIO SOBRE HORAS EXTRAS 50%,13o SALÁRIO SOBRE HORAS EXTRAS 50%,"1.648,55","47,35","1.695,90",NaN,NaN
FÉRIAS + 1/3 SOBRE HORAS EXTRAS 50%,FÉRIAS + 1/3 SOBRE HORAS EXTRAS 50%,"1.939,86","55,54","1.995,40",FÉRIAS + 1/3 SOBRE HORAS EXTRAS 50%,HORAS EXTRAS E REFLEXOS
FGTS 8%,FGTS 8%,"2.054,45","63,15","2.117,60",FGTS 8%,FGTS + MULTA DE 40%


## Completar DE/PARA

- .loc[] -> retorna um (ou mais) valor na notação [linha(s), coluna(s)]
- idxmax() -> índice do maior valor. Se houver empate, traz o primeiro

In [62]:
def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()

def compare_dfs(text, df):
    eq_ratio = df["DE"].apply(lambda x: similar(x, text))
    value = df.loc[ eq_ratio.idxmax(), "PARA" ]
    print("Search:", text, "Found:", value)
    return value


- ~ => not (nega o que vem à seguir)
- isna() => É vazio (numpy.nan)
- where() -> primeiro campo é a condição para manter, segundo campo é o valor (ou função) para trocar o que for falso
- onde(~(para é vazio), troquepor=similar_mais_proximo)

In [69]:
col_name = "Descrição do Bruto Devido ao Reclamante"

get_approx_value = lambda x: compare_dfs(x, dp)

df["PARA2"] = df[col_name].loc[ df["PARA"].isna() ].apply(get_approx_value)
df["PARA3"] = df["PARA2"].where( ~df["PARA2"].isna(), df["PARA"] )

df

Search: 13o SALÁRIO SOBRE HORAS EXTRAS 50% Found: HORAS EXTRAS E REFLEXOS


,Descrição do Bruto Devido ao Reclamante,Valor Corrigido,Juros,Total,DE,PARA,PARA2,PARA3
index,,,,,,,,
HORAS EXTRAS 50%,HORAS EXTRAS 50%,"24.995,90","715,34","25.711,24",HORAS EXTRAS 50%,HORAS EXTRAS E REFLEXOS,NaN,HORAS EXTRAS E REFLEXOS
13o SALÁRIO SOBRE HORAS EXTRAS 50%,13o SALÁRIO SOBRE HORAS EXTRAS 50%,"1.648,55","47,35","1.695,90",NaN,NaN,HORAS EXTRAS E REFLEXOS,HORAS EXTRAS E REFLEXOS
FÉRIAS + 1/3 SOBRE HORAS EXTRAS 50%,FÉRIAS + 1/3 SOBRE HORAS EXTRAS 50%,"1.939,86","55,54","1.995,40",FÉRIAS + 1/3 SOBRE HORAS EXTRAS 50%,HORAS EXTRAS E REFLEXOS,NaN,HORAS EXTRAS E REFLEXOS
FGTS 8%,FGTS 8%,"2.054,45","63,15","2.117,60",FGTS 8%,FGTS + MULTA DE 40%,NaN,FGTS + MULTA DE 40%


## Extrair valores

In [71]:
valores = df.groupby("PARA3").sum().loc[:, "Total"]
valores

KeyError: 'PARA3'